# Quantum Dice Roller

This is a quantum dice roller for D&D players! If you'd like to simply use it without understanding the code, simply move through each of the boxes to the bottom and follow the instructions given in the last box! To move through each of the boxes, press shift + enter.

In this code, we will use Qiskit to simulate a die roll on a quantum computer. To denote an N-die 'dN' is used, so N = 4 means "d4", N=6 means "d6" and so on.

The idea: a quantum register with n bits has 2^n possible outcomes that can be obtained after measurement, if they are all given an equal probability to be obtained. Thus, a quantum register with two qubits can have four possible outcomes: 00, 01, 10, and 11. The default state of a quantum register is 0, that is, all the qubits are zero. For two qubits, the default state is 00. However, if something called a 'Hadamard' operation is performed on each of the qubits, then each of the qubits become 0+1, with 0 and 1 having equal probabilities to be obtained upon measurement.

This is the Hadamard operation: 0 -> H -> 0+1. (Technically the 0 and 1 have a coefficient of 1/sqrt(2) to depict the probabilities, but we don't need to pay attention to that in this context. It suffices to understand that each of the two outcomes are equally likely.)

After applying a Hadamard to each qubit in a register with two qubits, all four outcomes are possible with equal probabilities. If each of the four possiblities is associated with each of the faces of a die, we essentially have a quantum d4. The result of the roll is the face that is associated with the highest number of counts when the circuit is created and performed hundreds of times.

Suppose the mapping between outcome and d4 faces are: 1 - 00, 2 - 01, 3 - 10, 4 - 11. If the result was, say: 
{'01': 262, '11': 245, '10': 257, '00': 260},

the result of the roll would be 2, because it has the largest counts associated with it at 262.

This method can be extended for the other dice as well. For instance, a register with three qubits can give 2^3 = 8 possible outcomes, which very neatly provides us with a d8. The more complex matter to consider is those dice that require something in beween, such as d6 or d12. That is addressed later.


The function run_QuantumCirc handles the quantum part of the code. A quantum circuit is created, a measurement is performed, and the result of the measurement in terms of counts is returned.

In [7]:
import qiskit
import cmd, sys
import math
import numpy as np

def run_QuantumCirc(n): # n - number of qubits
    qr = qiskit.QuantumRegister(n) # create quantum register with n qubits
    cr = qiskit.ClassicalRegister(n) # create classical register with n bits
    circ  = qiskit.QuantumCircuit(qr, cr) # create circuit with the two registers
    circ.h(qr) # perform Hadamard  on each qubit
    circ.measure(qr,cr) # each qubit is measured, and the outcome is either 0 or 1
    job = qiskit.execute(circ, qiskit.BasicAer.get_backend('qasm_simulator') ) # run on qasm_simulator
    #print(circ) # uncomment this if you would like to see the quantum circuit
    result = job.result().get_counts() # result is a dict, with key = classical bit outcomes, value = number of counts
    #print(result) # uncomment this if you would like to see the results of the measurements and the counts
    return result

The function run_dN handles the classical part of the code. First, it calculates the number of qubits required. Recall that for d4 we need 2 qubits (which is log(4) to the base 2), and for d8 we need 3 qubits (which is log(8) to the base 2). For d6, and other dice values that need something in between, we simply use the minimum number of qubits that gives us more possible states than 6 (that is, N).

Once equipped with the number of qubits, it calls run_QuantumCirc and examines the result as though it is 'rolling' until it gets a number less than 6. (Note that this method is a little clumsy and unelegant, but it allows each outcome to have equal probabilities of being rolled, which we want. Perhaps a better solution will come down the line!)

For instance, we use three qubits for a d6. Remember that three qubits means 8 possibilities:
1 - 000, 2 - 001, 3 - 010, 4 - 011, 5 - 100, 6 - 101, 7 - 110, 8 - 111

Suppose run_QuantumCirc returns the following: 
{'111': 144, '011': 132, '100': 136, '000': 110, '101': 126, '001': 135, '110': 120, '010': 121}

We see that the value 8 has the highest count at 144, but that is not a valid value for a d6. That option is eliminated, and the next highest count is considered: 136, which is associated with 5. Hence, the output of the d6 roll is 5. This method is followed for d10, d12, and d20 as well.

In [8]:
def run_dN(dN): # for dice dN
    if math.log(dN,2)%int(math.log(dN,2)) == 0: # checks whether log(N) to the base 2 is a whole number. used for d4, d8, d16
        n = int(math.log(dN,2)) # n = number of qubits
        result = run_QuantumCirc(n)
        max_res = max(result, key=result.get) # finds the max count
    else: # used for d6, d10, d12, d20
        n = int(math.log(dN,2))+1 # adds 1 to log(N) to the base 2 to get total number of qubits needed
        result = run_QuantumCirc(n)
        max_res = max(result, key=result.get)
        while(int(max_res,2) > dN-1): # find max that is less than N
            result.pop(str(max_res))
            max_res = max(result, key=result.get)
    return int(max_res,2)+1 # converts the binary result to a decimal 'roll', i.e. 000 = 0 in decimal but 0 + 1 as a roll, 010 = 2 in decimal, but 3 as a roll

In [14]:
#Click this box and press "shift + enter" to run it. 
#Enter your die choice. The code will by default run on IBM's quantum simulator, 
#unless you've chosen to run it on a real quantum computer in box number -- above.
text = input("Choose die (Options - 4, 6, 8, 10, 12, 20): ")
if int(text) in [4, 6, 8, 10, 12, 20]:
    print("Your roll is: " + str(run_dN(int(text))))
else:
    print('Not a valid die value!')

Choose die (Options - 4, 6, 8, 10, 12, 20): 12
Your roll is: 6
